In [27]:
# Setup
import csv
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [73]:
def feature_extraction(img, binary):
    
    pixel_is = img[binary == 255]
    
    mean_i = pixel_is.mean()
    std_i = pixel_is.std()

    gauss = cv2.GaussianBlur(img, (5,5), 0)
    lap = cv2.Laplacian(gauss, cv2.CV_64F)
    
    pixel_gs = lap[binary == 255]
    
    mean_g = pixel_gs.mean()
    std_g = pixel_gs.std()
          
    mag, ori = gradients(img)
            
# gauss_pos = [cv2.GaussianBlur(pos[i], (5,5), 0) for i in range(SAMPLE_SIZE)]
# gauss_neg = [cv2.GaussianBlur(neg[i], (5,5), 0) for i in range(SAMPLE_SIZE)]

# lap_pos = [cv2.Laplacian(gauss_pos[i], cv2.CV_64F) for i in range(SAMPLE_SIZE)]
# lap_neg = [cv2.Laplacian(gauss_neg[i], cv2.CV_64F) for i in range(SAMPLE_SIZE)]
    return mean_i, std_i, mean_g, std_g, mag, ori

In [33]:
''' First Pipeline : Greyscale -> Median Blur -> Otsu's Method + Inverse Threshold Binarization -> Erosion/Dilation  '''

def pip1(img, blurKernel=3, e_it=1, e_kern=5, d_it=1, d_kern=3):
    '''
    blurKernel: Size of median kernel (3x3, 5x5 ...)
    e_it: Number of erosion iterations 
    e_kern: Kernel size of erosion
    d_it: Number of dilation iterations
    d_kern: Kernel size of dilation
    '''
    
    # Linear Greyscale conversion (Y = 0.299*R + 0.587*G +0.114*B)
    g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Applies a median blur (blurKernek x blurKernel)
    blur = cv2.medianBlur(g_img, blurKernel)
    
    # Otsu's method finds dynamic threshold value, pixels are assigned either 0 or 255
    th_val, img_bin = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    # Erosion and Dilation
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (e_kern,e_kern))
    ero = cv2.erode(img_bin, kernel, iterations=e_it)
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (d_kern,d_kern))
    dil = cv2.dilate(ero, kernel, iterations=d_it)
    
    return dil


In [52]:
''' Second Pipeline : Greyscale -> Median Blur -> Morphological Transformation (Opening)  
    -> Otsu's Method + Inverse Threshold Binarization -> Erosion/Dilation '''

def pip2(img, blurKernel=3, open_k=5, open_its=(3, 1), e_it=2, e_kern=7, d_it=1, d_kern=3):
    '''
    blurKernel: Size of median kernel (3x3, 5x5 ...)
    e_it: Number of erosion iterations 
    e_kern: Kernel size of erosion
    d_it: Number of dilation iterations
    d_kern: Kernel size of dilation
    '''
    
    # Linear Greyscale conversion (Y = 0.299*R + 0.587*G +0.114*B)
    g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Applies a median blur (blurKernek x blurKernel)
    blur = cv2.medianBlur(g_img, blurKernel)
    
    # Opening (Erosion -> Dilation) on greyscale. 
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (open_k,open_k))
    erosion = cv2.erode(blur, kernel, iterations = open_its[0])
    dilation = cv2.dilate(erosion, kernel, iterations = open_its[1])
     
    # Otsu's method finds dynamic threshold value, pixels are assigned either 0 or 255
    th_val, img_bin = cv2.threshold(dilation, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    # Erosion and Dilation
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (e_kern,e_kern))
    ero = cv2.erode(img_bin, kernel, iterations=e_it)
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (d_kern,d_kern))
    dil = cv2.dilate(ero, kernel, iterations=d_it)
    
    return dil

In [74]:
''' Third Pipeline : Greyscale -> Median Blur -> Non-Linear Greyscale Transformation -> 
    Improved Otsu's Method + Inverse Threshold Binarization -> Erosion/Dilation '''

def pip3(img, blurKernel=3, open_k=5, open_its=(3, 1), e_it=1, e_kern=5, d_it=1, d_kern=5):
    '''
    blurKernel: Size of median kernel (3x3, 5x5 ...)
    e_it: Number of erosion iterations 
    e_kern: Kernel size of erosion
    d_it: Number of dilation iterations
    d_kern: Kernel size of dilation
    '''
    
    # Linear Greyscale conversion (Y = 0.299*R + 0.587*G +0.114*B)
    g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Applies a median blur (blurKernek x blurKernel)
    blur = cv2.medianBlur(g_img, blurKernel)
    
    # Opening (Erosion -> Dilation) on greyscale. 
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (open_k,open_k))
    erosion = cv2.erode(blur, kernel, iterations = 1)
    dilation = cv2.dilate(erosion, kernel, iterations = 1)
     
    # Otsu's method finds dynamic threshold value, pixels are assigned either 0 or 255
    th_val, img_bin = cv2.threshold(dilation, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Dynamically calculation Upper bound for the Non-linear threshold
    ub_lim = round(np.count_nonzero(img_bin)/(227*227), 4)
    
    # Non Linear Greyscale
    nl = nl_grayscale(blur, 0.02, ub_lim)

    # Improved Otsu's
    th_val, img_bin = cv2.threshold(dilation, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)    
    imp_otsu = improved_otsu(nl, th_val)
    
    # Erosion and Dilation
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (e_kern,e_kern))
    ero = cv2.erode(imp_otsu, kernel, iterations=e_it)
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (d_kern,d_kern))
    out = cv2.dilate(ero, kernel, iterations=d_it)
    
    return out

In [111]:
''' Fourth Pipeline : Greyscale -> Median Blur -> Non-Linear Greyscale Transformation -> 
    Otsu's Method on grey pixels only (no 0 or 255) + Inverse Threshold Binarization -> Erosion/Dilation '''

def pip4(img, blurKernel=3, open_k=5, open_its=(3, 1), e_it=1, e_kern=5, d_it=1, d_kern=5):
    '''
    blurKernel: Size of median kernel (3x3, 5x5 ...)
    e_it: Number of erosion iterations 
    e_kern: Kernel size of erosion
    d_it: Number of dilation iterations
    d_kern: Kernel size of dilation
    '''
    
    # Linear Greyscale conversion (Y = 0.299*R + 0.587*G +0.114*B)
    g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Applies a median blur (blurKernek x blurKernel)
    blur = cv2.medianBlur(g_img, blurKernel)
    
    # Opening (Erosion -> Dilation) on greyscale. 
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (open_k,open_k))
    erosion = cv2.erode(blur, kernel, iterations = 1)
    dilation = cv2.dilate(erosion, kernel, iterations = 1)
     
    # Otsu's method finds dynamic threshold value, pixels are assigned either 0 or 255
    th_val, img_bin = cv2.threshold(dilation, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    ub_lim = round(np.count_nonzero(img_bin)/(227*227), 4)
    
    # Non Linear Greyscale
    nl = nl_grayscale(blur, 0.02, ub_lim)
    nl_grey = nl[nl != 0]
    nl_grey = nl_grey[nl_grey != 255]

    #
    th_val, img_bin = cv2.threshold(nl_grey, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)    
    th_val, img_bin = cv2.threshold(nl, th_val, 255, cv2.THRESH_BINARY_INV)
    
    
    # Erosion and Dilation
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (e_kern,e_kern))
    ero = cv2.erode(img_bin, kernel, iterations=e_it)
    kernel = cv2.getStructuringElement(cv2.cv2.MORPH_ELLIPSE, (d_kern,d_kern))
    out = cv2.dilate(ero, kernel, iterations=d_it)
    
    return out

In [129]:
img_pos = cv2.imread("C:\\Users\\Rasmus\\Desktop\\Deep Learning\\concrete_data\\Positive\\{0:05d}.jpg".format(np.random.randint(1,10000)))
img_neg = cv2.imread("C:\\Users\\Rasmus\\Desktop\\Deep Learning\\concrete_data\\Negative\\{0:05d}.jpg".format(np.random.randint(1,20000)))

# img_pos = cv2.imread("C:\\Users\\Rasmus\\Desktop\\Deep Learning\\concrete_data\\Positive\\{0:05d}.jpg".format(6659))
# img_neg = cv2.imread("C:\\Users\\Rasmus\\Desktop\\Deep Learning\\concrete_data\\Negative\\{0:05d}.jpg".format(np.random.randint(1,20000)))

crack1 = pip1(img_pos)
no_crack1 = pip1(img_neg)

crack2 = pip2(img_pos)
no_crack2 = pip2(img_neg)

crack3 = pip3(img_pos)
no_crack3 = pip3(img_neg)

crack4 = pip4(img_pos)
no_crack4 = pip4(img_neg)

cv2.imshow("Raw Crack", img_pos)
cv2.imshow("Raw No Crack", img_neg)

cv2.imshow("Pipeline1 Crack", crack1)
cv2.imshow("Pipeline1 No Crack", no_crack1)

cv2.imshow("Pipeline2 Crack", crack2)
cv2.imshow("Pipeline2 No Crack", no_crack2)

cv2.imshow("Pipeline3 Crack", crack3)
cv2.imshow("Pipeline3 No Crack", no_crack3)

cv2.imshow("Pipeline4 Crack", crack4)
cv2.imshow("Pipeline4 No Crack", no_crack4)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Pipeline 1")
print("Crack -> (Mean, Std Dev) {}".format(feature_extraction(img_pos, crack1)))
print("No Crack -> (Mean, Std Dev)  {} ".format(feature_extraction(img_neg, no_crack1)))


print("Pipeline 2")
print("Crack -> (Mean, Std Dev) {}".format(feature_extraction(img_pos, crack2)))
print("No Crack -> (Mean, Std Dev)  {} ".format(feature_extraction(img_neg, no_crack2)))

print("Pipeline 3")
print("Crack -> (Mean, Std Dev) {}".format(feature_extraction(img_pos, crack3)))
print("No Crack -> (Mean, Std Dev)  {} ".format(feature_extraction(img_neg, no_crack3)))

print("Pipeline 4")
print("Crack -> (Mean, Std Dev) {}".format(feature_extraction(img_pos, crack4)))
print("No Crack -> (Mean, Std Dev)  {} ".format(feature_extraction(img_neg, no_crack4)))

C:\Users\Rasmus\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Rasmus\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Rasmus\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in ubyte_scalars


Pipeline 1
Crack -> (Mean, Std Dev) (93.55024364546293, 24.105391507683432, 1.457570569384082, 4.111231346696771, 37, 40)
No Crack -> (Mean, Std Dev)  (192.5274370411672, 7.8270298673465915, 0.7887278134002416, 2.6008989725584093, 36, 40) 
Pipeline 2
Crack -> (Mean, Std Dev) (99.07701123080449, 27.762191469126, 1.1011154404461763, 4.569383221343808, 37, 40)
No Crack -> (Mean, Std Dev)  (191.05830343877957, 9.693470609620638, 0.4963220639924629, 3.2252630701327227, 36, 40) 
Pipeline 3
Crack -> (Mean, Std Dev) (96.81588205067393, 25.424797890724378, 1.4454721530495276, 4.165436929095721, 37, 40)
No Crack -> (Mean, Std Dev)  (193.0990550466287, 7.710651314727455, 0.7972889329041842, 2.5949786921914755, 36, 40) 
Pipeline 4
Crack -> (Mean, Std Dev) (80.86816968675376, 18.07695609876684, 1.831015592077539, 4.004382199857822, 37, 40)
No Crack -> (Mean, Std Dev)  (186.2163947845407, 7.741270658533134, 1.2665614219727532, 2.897937886980571, 36, 40) 


In [64]:
def find_limits(img, lb, ub):
    w, h = img.shape
    im_arr = np.reshape(img, (w*h))
    
    im_arr = np.sort(im_arr)
    
    i_a = int(round(w*h * lb, 0))
    i_b = int(round(w*h * ub, 0))
    
    a = im_arr[i_a-1]
    b = im_arr[i_b-1]
    
    t = find_t(im_arr, i_a, i_b)
    
    return a, b, t

def find_t(arr, ia, ib):
    a, b = arr[ia], arr[ib]
    
    interval = arr[ia:ib+1]
    mean_i = interval.mean()
    
    t = (2*mean_i*(b - a)) / (255*(a+b))
    return t

def nl_grayscale(img, lb, ub):
    '''
    Non-Linear Greyscale Transformation
    
    Need's fixing: Upper Bound (ub) values .. if above 1.0 or below 0.02 it will mess up
    '''
    new_a = 0
    new_b = 255
    a, b, t = find_limits(img, lb, ub)
    w, h = img.shape
    

    im_out = img.copy()
    
    i = img<a
    im_out[i] = new_a
    
    i = img > b
    im_out[i] = new_b
    
    for i in range(w):
        for j in range(h):
            if img[i, j] < a and not img[i, j] > b:
                new_a + ((new_a - new_b) / (a**t - b**t)) * (img[i, j]**t - a**t)
                
    
    return im_out

In [63]:
def improved_otsu(img, thresh, ksize=7):

    w, h = img.shape
    x = ksize // 2
    y = ksize // 2
    im_out = img.copy()
    
    # STEP 1 -> 3
    
    while not x > w:
        while y < (h - ksize // 2):
            
            if not x == w:
                window = img[(x-3):(x+4),(y-3):(y+4)]
            else:
                window = img[(x-3):x,(y-3):(y+4)]
                
            var = window.std()
            

            if var < thresh:
                new_thresh, seg_window = cv2.threshold(window, thresh, 255, cv2.THRESH_BINARY_INV)
            else:
                new_thresh, seg_window = cv2.threshold(window, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                
                
            if not x == w:
                im_out[(x-3):(x+4),(y-3):(y+4)] = seg_window
            else:
                im_out[(x-3):x,(y-3):(y+4)] = seg_window
                
            y += ksize
            
        if not x == w:
            window = img[(x-3):(x+4),(y-3):y]
        else:
            window = img[(x-3):x,(y-3):y]
            
        var = window.std()
        
        if var < thresh:
            new_thresh, seg_window = cv2.threshold(window, thresh, 255, cv2.THRESH_BINARY_INV)
        else:
            new_thresh, seg_window = cv2.threshold(window, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        if not x == w:
            im_out[(x-3):(x+4),(y-3):y] = seg_window
        else:
            im_out[(x-3):x,(y-3):y] = seg_window
            
        x += ksize
        y = ksize // 2
    
    

            
    window = img[(x-3):x,(y-3):(y+4)]
    var = window.std()
    if var < thresh:
        new_thresh, seg_window = cv2.threshold(window, thresh, 255, cv2.THRESH_BINARY_INV)
    else:
        new_thresh, seg_window = cv2.threshold(window, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    im_out[(x-3):x,(y-3):(y+4)] = seg_window
    
    return im_out


In [72]:
def gradients(img):
    number_bins = 40

    ############ POSITIVE IMAGES ############################
    # Calculating the gradients
    sobelX_pos = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
    sobelY_pos = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)

    # Using vector tool to find the magnitudes and the angles
    mag, angle = cv2.cartToPolar(sobelX_pos, sobelY_pos, angleInDegrees=True)

    # Making the histograms
    hist_mag = np.histogram(mag, bins=number_bins)
    hist_angle = np.histogram(angle, bins=number_bins)
    
    
    # This Is the Version where we compress the values into bins
    un_mag = len(np.unique(hist_mag[0]))
    un_angle = len(np.unique(hist_angle[0]))
    
    # This is the version where we do not compress values into bins
#     un_mag = len(np.unique(mag))
#     un_angle = len(np.unique(angle))
    
    return un_mag, un_angle